<a href="https://colab.research.google.com/github/Dkaattae/stock_markets_analysis/blob/main/stock_market_analysis_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# IMPORTS
import numpy as np
import pandas as pd
import requests


#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt



# Question 1, withdrawn ipos by company type


In [2]:
from io import StringIO

def get_ipos_withdrawn() -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for withdrawn.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [3]:
withdrawn_ipo = get_ipos_withdrawn()
withdrawn_ipo

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-
...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-
96,CHO,Chobani Inc.,-,-
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231
98,GLGX,"Gerson Lehrman Group, Inc.",-,-


In [4]:
import re

def find_company_class(company_name: str) -> str:
  company_words = re.findall(r'\b\w+\b', company_name.lower())
  if 'acquisition' in company_words and 'corp' in company_words:
    return 'Acq.Corp'
  elif 'acquisition' in company_words and 'corporation' in company_words:
    return 'Acq.Corp'
  elif 'inc' in company_words:
    return 'Inc'
  elif 'group' in company_words:
    return 'Group'
  elif 'ltd' in company_words:
    return 'Ltd'
  elif 'limited' in company_words:
    return 'Ltd'
  elif 'holdings' in company_words:
    return 'Holdings'
  else:
    return 'Other'

# find_company_class('Thimble Point Acquisition Corp. II')
withdrawn_ipo['Company Class'] = withdrawn_ipo['Company Name'].apply(find_company_class)

In [5]:
def find_avg_price(price_range: str) -> float:
  price_ranges = price_range.split('-')
  price_ranges = [price.replace("$", "").strip() for price in price_ranges]
  if price_ranges[0] == '':
    return 'None'
  elif len(price_ranges) == 2:
    return (float(price_ranges[0]) + float(price_ranges[1])) / 2
  else:
    return float(price_ranges[0])

withdrawn_ipo['Avg. price'] = withdrawn_ipo['Price Range'].apply(find_avg_price)

In [6]:
def extract_int(s):
    match = re.search(r'\d+', s)
    return int(match.group()) if match else None

withdrawn_ipo['Shares Offered'] = withdrawn_ipo['Shares Offered'].apply(extract_int)

In [ ]:
withdrawn_ipo

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price,Withdrawn Value
0,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5,7000000.0
1,AURN,"Aurion Biotech, Inc.",-,NaN,Inc,None,NaN
2,ROTR,"PHI Group, Inc.",-,NaN,Inc,None,NaN
3,ONE,One Power Company,-,NaN,Other,None,NaN
4,HPOT,The Great Restaurant Development Holdings Limited,$4.00 - $6.00,1400000.0,Holdings,5.0,7000000.0
...,...,...,...,...,...,...,...
94,FHP,"Freehold Properties, Inc.",-,NaN,Inc,None,NaN
95,CHO,Chobani Inc.,-,NaN,Inc,None,NaN
96,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231.0,Inc,19.5,600000004.5
97,GLGX,"Gerson Lehrman Group, Inc.",-,NaN,Inc,None,NaN


In [7]:
def find_value(row):
  if pd.notnull(row['Shares Offered']) and row['Avg. price'] != 'None':
    return row['Shares Offered'] * row['Avg. price']
  else:
    return None

withdrawn_ipo['Withdrawn Value'] = withdrawn_ipo.apply(find_value, axis=1)

In [8]:
withdrawn_ipo[withdrawn_ipo['Company Class']=='Other']

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price,Withdrawn Value
4,ONE,One Power Company,-,NaN,Other,None,NaN
9,KMCM,Key Mining Corp.,$2.25,4444444.0,Other,2.25,9999999.0
53,CLLB,"CoLabs Intâl, Corp.",$4.50,1300000.0,Other,4.5,5850000.0
74,TSIV,Twelve Seas Investment Company IV TMT,$10.00,20000000.0,Other,10.0,200000000.0
86,FSPR,Four Springs Capital Trust,$13.00 - $15.00,18000000.0,Other,14.0,252000000.0
99,HCG,hear.com N.V.,$17.00 - $20.00,16220000.0,Other,18.5,300070000.0


In [9]:
withdrawn_ipo['Withdrawn Value'].notna().sum()

np.int64(71)

In [10]:
withdrawn_ipo.groupby('Company Class')['Withdrawn Value'].sum()

,Withdrawn Value
Company Class,
Acq.Corp,4.021000e+09
Group,3.378750e+07
Holdings,7.500000e+07
Inc,2.257164e+09
Ltd,5.497346e+08
Other,7.679200e+08


question 1 answer,   
Acquisition Corp with $4021 million withdrawn value

# Question 2, median sharpe ratio for 2024 ipos

In [11]:
def get_ipos_by_year(year: int) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [20]:
ipos_2024 = get_ipos_by_year(2024)
ipos_2024['IPO Date'] = pd.to_datetime(ipos_2024['IPO Date'], format='mixed')
ipos_2024_first5mo = ipos_2024[ipos_2024['IPO Date'] < '2024-06-01']
ipos_2024_first5mo = ipos_2024_first5mo.copy()
# ipos_2024_first5mo['IPO Price'] = ipos_2024_first5mo['IPO Price'].replace('-', np.nan)
ipos_2024_first5mo['IPO Price'] = ipos_2024_first5mo['IPO Price'].str.replace('$', '', regex=False)
ipos_2024_first5mo['IPO Price'] = pd.to_numeric(ipos_2024_first5mo['IPO Price'], errors='coerce')

In [23]:
# Convert "Current" column
ipos_2024_first5mo['Current'] = pd.to_numeric(ipos_2024_first5mo['Current'].str.replace('$', ''), errors='coerce')

# Convert 'Return' to numeric format (percentage)
ipos_2024_first5mo['Return'] = pd.to_numeric(ipos_2024_first5mo['Return'].str.replace('%', ''), errors='coerce') / 100



In [24]:
ipos_2024_first5mo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77 entries, 148 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      77 non-null     datetime64[ns]
 1   Symbol        77 non-null     object        
 2   Company Name  77 non-null     object        
 3   IPO Price     75 non-null     float64       
 4   Current       77 non-null     float64       
 5   Return        75 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 4.2+ KB


In [25]:
import time
import datetime

stocks_df = pd.DataFrame({'A' : []})
ALL_TICKERS = ipos_2024_first5mo['Symbol']

for i,ticker in enumerate(ALL_TICKERS):
  # print(i,ticker)

  ticker_obj = yf.Ticker(ticker)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  for i in [1,3,5,21,63,252]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)
  historyPrices['Sharpe'] = (historyPrices['growth_252d'] - 0.045) / historyPrices['volatility']

  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

In [26]:
stock_20250606 = stocks_df[stocks_df.Date == datetime.date(2025,6,6)]
stock_20250606.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,Month,Weekday,growth_1d,growth_3d,growth_5d,growth_21d,growth_63d,growth_252d,volatility,Sharpe
count,77.000000,77.000000,77.000000,77.000000,7.700000e+01,77.0,77.0,77.0,77.0,77.0,77.000000,77.000000,77.000000,77.000000,77.000000,73.000000,77.000000,73.000000
mean,16.512838,16.842268,16.145447,16.559969,1.363757e+06,0.0,0.0,2025.0,6.0,4.0,1.015585,1.013348,1.050006,1.168460,1.250822,1.227948,20.595412,0.297523
std,25.617474,26.114982,24.970946,25.725647,5.221262e+06,0.0,0.0,0.0,0.0,0.0,0.059093,0.131618,0.246880,0.603348,1.215962,1.480237,37.312322,0.523190
min,0.000500,0.000600,0.000400,0.000600,0.000000e+00,0.0,0.0,2025.0,6.0,4.0,0.862573,0.565134,0.473585,0.552434,0.230769,0.024970,0.004299,-0.079677
25%,1.210000,1.320000,1.161000,1.290000,3.810000e+04,0.0,0.0,2025.0,6.0,4.0,0.995994,0.980422,0.979379,0.930000,0.801242,0.293510,1.368371,0.040265
50%,5.310000,5.355000,5.289000,5.320000,1.379000e+05,0.0,0.0,2025.0,6.0,4.0,1.004785,1.000952,1.008596,1.018499,1.007111,0.763188,3.991950,0.083768
75%,20.430000,20.440001,19.969999,20.410000,5.075000e+05,0.0,0.0,2025.0,6.0,4.0,1.025957,1.040819,1.052419,1.204566,1.262270,1.446667,19.726044,0.331967
max,113.629997,121.290001,112.769997,121.290001,4.401273e+07,0.0,0.0,2025.0,6.0,4.0,1.240000,1.808333,2.520325,5.139159,9.624243,8.097413,188.035427,2.835668


question 2 answer,   
median sharpe ratio: 0.082238

note: the median of 252d return should be 1.15, i got 1.21.
could be related to two tickers without ipo price.

bonus question:  growth_252d is the same as sharpe ratio?   
of course no. that is why we come up with metric sharpe ratio, it is a metric measuring both return and risk. while 1yr return cannot grasp risk.

In [27]:
stock_20250606.sort_values(by='growth_252d', ascending=False).head(10)['Ticker']

,Ticker
20630,JL
23407,ROMA
254,NAKA
14544,UMAC
3154,NNE
4808,RBRK
16892,AHR
18215,AS
7388,SUPX
4525,MRX


In [28]:
stock_20250606.sort_values(by='Sharpe', ascending=False).head(10)['Ticker']

,Ticker
11214,BKHA
21292,JVSA
17194,LEGT
10638,IBAC
5093,NCI
14879,HLXB
8556,MNDR
13883,DYCQ
12442,INTJ
20630,JL


Question 3, Fixed Month Holding Strategy.  

In [33]:
stocks_df = stocks_df[~stocks_df['Ticker'].isin(['NAKA','SUPX'])]

In [34]:
for i in range(12):
  d = 21 * (i+1)
  stocks_df['future_close_'+str(i+1)+'m'] = stocks_df.groupby('Ticker')['Close'].shift(-d)
  stocks_df['future_growth_'+str(i+1)+'m'] = stocks_df['future_close_'+str(i+1)+'m'] / stocks_df['Close']

In [35]:
stock_growth = pd.DataFrame()
stock_mindate = stocks_df.groupby('Ticker')['Date'].min()
stock_growth = pd.merge(stock_mindate.reset_index(), stocks_df, on=['Ticker', 'Date'])

In [36]:
features = ['Ticker']
for i in range(12):
  features.append('future_growth_'+str(i+1)+'m')
stock_growth_describe = stock_growth[features].describe()

In [37]:
stock_growth_describe

,future_growth_1m,future_growth_2m,future_growth_3m,future_growth_4m,future_growth_5m,future_growth_6m,future_growth_7m,future_growth_8m,future_growth_9m,future_growth_10m,future_growth_11m,future_growth_12m
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000,74.000000,74.000000,71.000000
mean,0.927259,0.940544,0.833824,0.825086,0.803768,0.864185,0.847149,0.832983,0.892996,0.917945,0.882534,0.900861
std,0.346261,0.574545,0.409763,0.401969,0.488348,0.653179,0.712943,0.762423,0.938269,0.911431,0.863183,0.892906
min,0.098947,0.073800,0.060947,0.045368,0.054109,0.061432,0.044086,0.043103,0.033144,0.037769,0.023674,0.038947
25%,0.778984,0.685815,0.511212,0.517233,0.448403,0.384560,0.296870,0.208677,0.244048,0.242424,0.264661,0.229211
50%,0.977000,1.000000,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.833357,0.772592,0.717585,0.634667
75%,1.046509,1.154013,1.069085,1.134300,1.016381,1.093948,1.114468,1.082433,1.055767,1.200791,1.106769,1.136392
max,2.646505,4.874759,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,5.352601,4.445545,4.849711


question 3 answer,   
future growth 2 m has the max mean.   
note: this growth should compare with index growth, which is access return.

another note: there are two companies that does not have an IPO price, it is because they changed ticker. the IPO price could be tracked by changing name history. or could be tracked through edgar SEC filings using CIK.
those companies should not be ignored, as changing ticker indicates big company stategy change, M&A for example, could impact huge on stock price.

take 2024 for example, the two missing companies are 'NAKA' and 'SUPX'. they all have significant returns during one year period, more than doubled, while most IPO company cannot produce positive returns.

#Question 4, RSI based trading strategy

In [1]:
!pip install pandas-ta

In [4]:
# copy code snippet

# https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

# You're required to add EU_STOCKS and INDIA_STOCS
# https://companiesmarketcap.com/european-union/largest-companies-in-the-eu-by-market-cap/
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

# https://companiesmarketcap.com/india/largest-companies-in-india-by-market-cap/
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

ALL_TICKERS = US_STOCKS  + EU_STOCKS + INDIA_STOCKS

stocks4_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  # print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks4_df.empty:
    stocks4_df = historyPrices
  else:
    stocks4_df = pd.concat([stocks4_df, historyPrices], ignore_index=True)

In [5]:
def get_ticker_type(ticker:str, us_stocks_list, eu_stocks_list, india_stocks_list):
  if ticker in us_stocks_list:
    return 'US'
  elif ticker in eu_stocks_list:
    return 'EU'
  elif ticker in india_stocks_list:
    return 'INDIA'
  else:
    return 'ERROR'

stocks4_df['ticker_type'] = stocks4_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS))

In [6]:
stocks4_df['Date'] = pd.to_datetime(stocks4_df['Date'])

In [43]:
!pip install numpy==1.24.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 58.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.3.4 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 w

In [7]:
import pandas_ta as ta

In [8]:
# momentum indicators: ADX, ADXR, APO, AROON, AROONOSC, BOP, CCI, CMO, DX, MACD, MACDEXT, MACDFIX, MFI,
#   MINUS_DI, MOM, PLUS_DI, PLUS_DM, PPO, ROC, ROCP, ROCR, ROCR100, RSI, STOCH, STOCHF,
#   STOCHRSI, TRIX, ULTOSC, WILLR
momentum_technical_indicators = [
    {"kind": "rsi", "length": 14},
    {"kind": "macd"},
    {"kind": "mom", "length": 14},
]
stocks4_df['rsi'] = ta.rsi(stocks4_df['Close'], length=14)
# stocks4_df['macd'] = ta.macd(stocks4_df['Close'])
stocks4_df['mom'] = ta.mom(stocks4_df['Close'], length=14)

In [9]:
rsi_threshold = 25
selected_df = stocks4_df[
    (stocks4_df['rsi'] < rsi_threshold) &
    (stocks4_df['Date'] >= '2000-01-01') &
    (stocks4_df['Date'] <= '2025-06-01')
]


In [10]:
investing_amount = 1000
net_income = investing_amount * (selected_df['growth_future_30d'] - 1).sum()

In [11]:
net_income

104500.89313875602